In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys,os
sys.path.append('..')
from backtester import matlab, backtester
from backtester.analysis import *
from backtester.strategy import StrategyBase, OptParam, OptParamArray
from backtester.swarms.ranking import SwarmRanker
from backtester.swarms.rebalancing import SwarmRebalance
from backtester.swarms.filters import SwarmFilter
from backtester.costs import CostsManagerEXOFixed
from backtester.exoinfo import EXOInfo
from backtester.swarms.rankingclasses import *
from backtester.swarms.swarm import Swarm

import statsmodels.tsa.api as smt
from sklearn import neighbors

from pandas.tseries.offsets import BDay

import pandas as pd
import numpy as np
import scipy

In [ ]:
from scripts.settings import *
try:
    from scripts.settings_local import *
except:
    pass

from exobuilder.data.exostorage import EXOStorage

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)


exo_filter = '*'     # All 
#exo_filter = 'ES_'  # ES only
exo_filter = 'CL'  # ES Collars (incl Bearish, Bullish, BW and vanilla)

[print(exo) for exo in storage.exo_list(exo_filter)];

In [ ]:
class Strategy_DMI(StrategyBase):
    name = 'Strategy_DMI'


    def __init__(self, strategy_context):
        # Initialize parent class
        super().__init__(strategy_context)
   
    def calc_entry_rules(self, H, L, C, di_ema_period, atr_period, pct_rank_value, adx_period, rules_index):
        i = 100
        px_ser = self.data.exo
        '''
        https://ru.tradingview.com/stock-charts-support/index.php/Directional_Movement_(DMI)

        Calculating the DMI can actually be broken down into two parts. 
        First, calculating the +DI and -DI, and second, calculating the ADX.

        To calculate the +DI and -DI you need to find the +DM and -DM (Directional Movement). 
        +DM and -DM are calculated using the High, Low and Close for each period. 
        You can then calculate the following:

        Current High - Previous High = UpMove
        Current Low - Previous Low = DownMove

        If UpMove > DownMove and UpMove > 0, then +DM = UpMove, else +DM = 0
        If DownMove > Upmove and Downmove > 0, then -DM = DownMove, else -DM = 0

        Once you have the current +DM and -DM calculated, the +DM and -DM lines can be 
        calculated and plotted based on the number of user defined periods.

        +DI = 100 times Exponential Moving Average of (+DM / Average True Range)
        -DI = 100 times Exponential Moving Average of (-DM / Average True Range)

        Now that -+DX and -DX have been calculated, the last step is calculating the ADX.

        ADX = 100 times the Exponential Moving Average of the Absolute Value of (+DI- -DI) / (+DI + -DI)'''
        
        #
        # Since there is no HL data, i will use a rolling max of C as High and rolling min of C as Low
        #
        hl_data_available = False

        #

        if hl_data_available == False:

            H = C.rolling(2).max().dropna()
            L = C.rolling(2).min().dropna()

            C = C[H.index]

        if hl_data_available == True:
            H = H
            L = L
   
        
        prev_high = H.shift(1)
        prev_low = L.shift(1)

        dm_pos = pd.Series(0.0, index=C.index)
        dm_neg = pd.Series(0.0, index=C.index)

        atr = ATR(H,L,C, atr_period)

        for i in range(C.size):
            upmove = H[i] - prev_high[i]
            downmove =  prev_low[i] - L[i]
            #print(upmove > downmove)

            if (upmove > downmove) & (upmove > 0):
                dm_pos[i] = upmove
                #print(dm_pos.iloc[i])

            if (downmove > upmove) & (downmove > 0):
                dm_neg[i] = downmove

        di_pos = ((dm_pos.ewm(di_ema_period).mean() / atr).ewm(di_ema_period).mean()) * 100
        di_neg = ((dm_neg.ewm(di_ema_period).mean() / atr).ewm(di_ema_period).mean()) * 100
        
        di_pos_pctrank = di_pos.rank(pct=True)
        di_neg_pctrank = di_pos.rank(pct=True)
        
        adx = ((di_pos - di_neg) / (di_pos + di_neg)).abs().ewm(adx_period).mean() * 100

        #adx_expandingquantile20 = adx.expanding(60).quantile(0.2)
        #adx_expandingquantile80 = adx.expanding(60).quantile(0.8)
        
        adx_pctchange5_pctrank = adx.pct_change(5).rank(pct=True)
        
        
        if rules_index == 0:
            return di_pos > di_neg
        
        if rules_index == 1:
            return di_neg > di_pos
        
        if rules_index == 2:
            return CrossDown(di_pos,di_neg)
        
        if rules_index == 3:
            return CrossUp(di_pos,di_neg)
        
        if rules_index == 4:
            return CrossUp(di_neg,di_pos)
        
        if rules_index == 5:
            return CrossDown(di_neg,di_pos)
        
        if rules_index == 6:
            return di_pos_pctrank >= pct_rank_value
        
        if rules_index == 7:
            return di_pos_pctrank <= pct_rank_value
        
        if rules_index == 8:
            return di_neg_pctrank >= pct_rank_value
        
        if rules_index == 9:
            return di_neg_pctrank <= pct_rank_value
        
        
        if rules_index == 10:
            return (di_pos > di_neg) & (adx_pctchange5_pctrank >= pct_rank_value)
        
        if rules_index == 11:
            return (di_pos > di_neg) & (adx_pctchange5_pctrank <= pct_rank_value)
        
        
        if rules_index == 12:
            return (di_neg > di_pos) & (adx_pctchange5_pctrank >= pct_rank_value)
        
        if rules_index == 13:
            return (di_neg > di_pos) & (adx_pctchange5_pctrank <= pct_rank_value)
    
    def calculate(self, params=None, save_info=False):
    #
    #
    #  Params is a tripple like (50, 10, 15), where:
    #   50 - slow MA period
    #   10 - fast MA period
    #   15 - median period
    #
    #  On every iteration of swarming algorithm, parameter set will be different.
    #  For more information look inside: /notebooks/tmp/Swarming engine research.ipynb
    #

        if params is None:
            # Return default parameters
            direction, di_ema_period, atr_period, pct_rank_value, adx_period, rules_index, period_median = self.default_opts()
        else:
            # Unpacking optimization params
            #  in order in self.opts definition
            direction, di_ema_period, atr_period, pct_rank_value, adx_period, rules_index, period_median = params

        # Defining EXO price
        px = self.data.exo
                
        # Median based trailing stop
        trailing_stop = px.rolling(period_median).median().shift(1)

        H = L = C = px
        
        # Enry/exit rules
        entry_rule = self.calc_entry_rules(H, L, C, di_ema_period, atr_period, pct_rank_value, adx_period, rules_index)

        if direction == 1:
            exit_rule = (CrossDown(px, trailing_stop))  # Cross down for longs

        elif direction == -1:
            exit_rule = (CrossUp(px, trailing_stop))
            

        # Swarm_member_name must be *unique* for every swarm member
        # We use params values for uniqueness
        swarm_member_name = self.get_member_name(params)

        #
        # Calculation info
        #
        calc_info = None
        if save_info:
            calc_info = {'trailing_stop': trailing_stop}

        return swarm_member_name, entry_rule, exit_rule, calc_info

## Script settings

In [ ]:
STRATEGY_CONTEXT = {
    'strategy': { 
        'class': Strategy_DMI,
        'exo_name': 'ZN_PutSpread',        # <---- Select and paste EXO name from cell above
        'exo_storage': storage,          
        'opt_params': [
                        #OptParam(name, default_value, min_value, max_value, step)
                        OptParamArray('Direction', [-1]),
                        OptParam('DI EMA period', 1, 5, 30, 10),
                        OptParamArray('ATR period', [14]),
                        OptParamArray('Pct. rank values', [0.05, 0.1, 0.9, 0.95]),
                        OptParamArray('ADX', [20,40]),
                        OptParamArray('Rules index', np.arange(13)),
                        OptParam('MedianPeriod', 1, 5, 30, 10),
                        
            ],
    },
    'swarm': {
        'members_count': 2,
        'ranking_class': RankerBestWithCorrel(window_size=-1, correl_threshold=0.5),
        'rebalance_time_function': SwarmRebalance.every_friday,

    },
    'costs':{
        'manager': CostsManagerEXOFixed,
        'context': {
            'costs_options': 3.0,
            'costs_futures': 3.0,
        }
    }
}

# Backtest class based strategy

In [ ]:
smgr = Swarm(STRATEGY_CONTEXT)
smgr.run_swarm()
smgr.pick()

# Saving results to swarms directory
smgr.save('./swarms/')

### WARNING! Loading swarm from file (don't run next cell if you want new swarm instance)

In [ ]:
smgr.strategy.data.exo.plot()

In [ ]:
figsize(10,10)
smgr.picked_equity.plot(label='Picked swarm equity');
smgr.raw_equity.plot(label='Average swarm equity');
legend(loc=2);

In [ ]:
smgr.raw_swarm.plot(legend=False)

(smgr.strategy.data.exo*-1).plot(linewidth=3, color='red')

In [ ]:
smgr.picked_swarm.plot()

smgr.picked_swarm.sum(1).plot(label='smgr.picked_swarm-sum')
smgr.picked_equity.plot(label='Picked swarm equity')

# Swarm exposure

In [ ]:
smgr.picked_exposure.sum(axis=1).rolling(10).mean().plot()

# Swarm statistics

#### Non-picked swarm stats

In [ ]:
smgr.picked_stats

## Exo information

In [ ]:
smgr.strategy.exoinfo.exo_info

In [ ]:
smgr.strategy.exoinfo.data.exo.plot()

### Global filter information (obsolete)


## Costs information (per 1-exo unit)

In [ ]:
figsize(10,5)
smgr.strategy.costs.plot()

## Margin graphs

### EXO Margin (per 1 EXO unit)

In [ ]:
smgr.strategy.exoinfo.margin().plot()

# Saving results

In [ ]:
smgr.save('./swarms/')

In [ ]:
smgr.raw_swarm#.plot()